In [ ]:
# default_exp main

# rfpy - CFRS Rfeye Node Logger binaries parser

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically

In [ ]:
#export
from rfpy.constants import SPECTRAL_BLOCKS
from typing import *
import os
from fastcore.xtras import Path
from fastcore.script import call_parse, Param, store_true
from rfpy.parser import *
import pandas as pd
from rfpy.utils import *
from rich.progress import Progress
from loguru import logger

In [ ]:
#export
def read_meta(filename):
    ext = filename.suffix
    if ext == ".csv":
        df = pd.read_csv(filename)
    elif ext == ".xlsx":
        df = pd.read_excel(filename, engine="openpyxl")
    elif ext == ".fth":
        df = pd.read_feather(filename)
        if "wallclock_datetime" in df.columns:
            df.set_index("wallclock_datetime", inplace=True)
    else:
        raise ValueError(f"Extension {ext} not implemented")
    return df


@logger.catch
@call_parse
def process_bin(
    entrada: Param("Diretório contendo arquivos .bin", str),
    saida: Param("Diretório para salvar os arquivos de saída", str),
    recursivo: Param("Buscar arquivos de maneira recursiva?", store_true) = False,
    pastas: Param("Limitar a busca às pastas", Iterable[str]) = None,
    meta: Param("Extrair e Salvar os metadados?", store_true) = False,
    levels: Param("Extrair e Salvar os níveis de Espectro?", store_true) = False,
    meta_ext: Param("Extensão do arquivo de metadados", str) = ".fth",
    levels_ext: Param("Extensão do arquivo de níveis", str) = ".fth",
    substituir: Param(
        "Reprocessar e substituir arquivos existentes?", store_true
    ) = False,
):

    lista_bins = get_files(
        entrada, extensions=[".bin"], recurse=recursivo, folders=pastas
    )
    parsed_bins = {}
    meta_path = Path(f"{saida}/meta")
    levels_path = Path(f"{saida}/levels")
    meta_path.mkdir(exist_ok=True, parents=True)
    levels_path.mkdir(exist_ok=True, parents=True)

    log_meta = Path(f"{saida}/log_meta.txt")
    log_levels = Path(f"{saida}/log_levels.txt")
    if substituir:
        done_meta = set()
        done_levels = set()
    else:

        done_meta = (
            set(log_meta.read_text().split("\n")) if log_meta.exists() else set()
        )
        done_levels = (
            set(log_levels.read_text().split("\n")) if log_levels.exists() else set()
        )

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(
        [f.name for f in lista_bins],
        style="bold white",
        overflow="fold",
        justify="left",
    )
    if not lista_bins:
        return

    erro_formato = False
    erro_novo = False

    with Progress(transient=True, auto_refresh=False) as progress:
        bins = progress.track(
            lista_bins,
            total=len(lista_bins),
            description="[green]Processando Blocos Binários",
        )
        for file in bins:
            progress.console.print(f"[green]Processando Blocos de: [yellow]{file.name}")
            parsed_bins[file.stem] = parse_bin(file)
            progress.refresh()
        if meta:
            lista = [f for f in lista_bins if f.name not in done_meta]
            if not lista:
                erro_novo = True
            else:
                bins = progress.track(
                    lista, total=len(lista), description="[cyan]Exportando Metadados"
                )
                for file in bins:
                    progress.console.print(
                        f"[cyan]Extraindo Metadados de: [yellow]{file.name}"
                    )
                    export_meta(
                        file.stem, parsed_bins[file.stem], meta_path, ext=meta_ext
                    )
                    done_meta.add(file.name)
                    progress.refresh()
        if levels:
            if meta_ext == ".csv":
                erro_formato = True
                lista = []
            else:
                lista = [f for f in lista_bins if f.name not in done_levels]
                if not lista:
                    erro_novo = True
                else:
                    bins = progress.track(
                        lista,
                        total=len(lista),
                        description="[orange]Exportando Dados de Espectro",
                    )
                    for file in bins:
                        progress.console.print(
                            f"[red]Extraindo Espectro de: [yellow]{file.name}"
                        )
                        meta_index = []
                        blocks = parsed_bins[file.stem]["blocks"]
                        for (tipo, tid) in blocks.keys():
                            if tipo not in SPECTRAL_BLOCKS:
                                continue
                            meta_file = Path(
                                f"{meta_path}/{file.stem}-B_{tipo}_TId_{tid}{meta_ext}"
                            )
                            if not meta_file.exists():
                                export_meta(
                                    file.stem,
                                    parsed_bins[file.stem],
                                    meta_path,
                                    ext=meta_ext,
                                )
                                done_meta.add(file.name)
                            meta_df = read_meta(meta_file)
                            meta_index.append(meta_df.index.tolist())
                        export_level(
                            file.stem,
                            parsed_bins[file.stem],
                            levels_path,
                            ext=levels_ext,
                            index=meta_index,
                        )
                        done_levels.add(file.name)
    if erro_formato:
        console.print(
            ":warning: Dados espectrais em .csv não são suportados pela explosão no tamanho de arquivo :exclamation:"
        )
        console.print("Sugerimos o formato:point_right: [red].fth")
    elif erro_novo:
        console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
        console.print(
            ":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:"
        )
    else:
        log_meta.write_text("\n".join(sorted(list(done_meta))))
        log_levels.write_text("\n".join(sorted(list(done_levels))))
        console.print("kbô :satisfied:")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
